# 4주차 과제

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 1번 문제

In [155]:
train = pd.read_csv('Churn_train.csv')
test = pd.read_csv("Churn_test.csv")

In [156]:
train.groupby('Geography').Exited.mean()

Geography
France     0.160236
Germany    0.332462
Spain      0.169272
Name: Exited, dtype: float64

In [157]:
train['geo_pred'] = [1 if g == 'Germany' else 0 for g in train['Geography']]

In [158]:
train = pd.get_dummies(train)

In [161]:
s = pd.DataFrame(scaler.fit_transform(train.iloc[:, -3:]), columns = ['a', 'b', 'c'])

In [163]:
s['a'].unique()

array([ 1.74445599, -0.57324462])

In [153]:
train.iloc[:, -3:]

,Geography_Spain,Gender_Female,Gender_Male
0,1,1,0
1,0,1,0
2,0,1,0
3,1,1,0
4,0,1,0
...,...,...,...
7495,1,0,1
7496,1,0,1
7497,0,0,1
7498,0,1,0


In [6]:
train.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)
test.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)

## 1. Age가 이상치인 데이터의 Balance 표준편차를 구하시오.

In [8]:
import numpy as np

In [9]:
q1 = np.percentile(train['Age'], 25)
q3 = np.percentile(train['Age'], 75)

In [11]:
q1, q3

(32.0, 44.0)

In [12]:
iqr = q3 - q1

In [13]:
l_b = q1 - 1.5 * iqr
u_b = q3 + 1.5 * iqr

In [16]:
train[(train['Age'] < l_b) | (train['Age'] > u_b)].Balance.std()

62552.16900161373

## 2. Credit Score에 대해 표준화를 실시하고 0.65 보다 큰 데이터의 수를 구하시오.

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()

In [20]:
scaled = scaler.fit_transform(train[['CreditScore']])

In [22]:
train['scaled_cs'] = scaled

In [25]:
train[train.scaled_cs > 0.65].shape[0]

3075

## 3. precision 점수를 기준으로 모델링 진행 후 예측값을 도출 후 submission을 제출하시오.

- 종속변수 : Exited

In [27]:
train.Geography.unique()

array(['Spain', 'France', 'Germany'], dtype=object)

In [30]:
train['Gender'] = [1 if g == 'Male' else 0 for g in train['Gender']]
test['Gender'] = [1 if g == 'Male' else 0 for g in test['Gender']]

In [32]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [46]:
X = train.drop(['Exited', 'scaled_cs'], axis = 1)
y = train.Exited

In [47]:
target = test.drop('Exited', axis = 1)

In [40]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score

In [38]:
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)

In [74]:
lgbm_pred = np.zeros((target.shape[0]))

In [75]:
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42)
    
    lgbm.fit(tr_x, tr_y)
    
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    val_pred = val_pred > 0.3
    val_pred = val_pred.astype(int)
    
    val_score = precision_score(val_y, val_pred)

    print(val_score)
    
    fold_pred = lgbm.predict_proba(target)[:, 1] / 10
    lgbm_pred = fold_pred

0.625
0.5879120879120879
0.6143790849673203
0.5934065934065934
0.61875
0.6358024691358025
0.5822784810126582
0.6035502958579881
0.611764705882353
0.6441717791411042


In [76]:
xgb_pred = np.zeros((target.shape[0]))

In [77]:
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb = XGBClassifier(random_state = 42)
    
    xgb.fit(tr_x, tr_y)
    
    val_pred = xgb.predict_proba(val_x)[:, 1]
    val_pred = val_pred > 0.3
    val_pred = val_pred.astype(int)
    
    val_score = precision_score(val_y, val_pred)

    print(val_score)
    
    fold_pred = xgb.predict_proba(target)[:, 1] / 10
    xgb_pred = fold_pred

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:30:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5925925925925926
[22:30:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5555555555555556
[22:30:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6266666666666667
[22:30:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3

In [78]:
lgbm_pred

array([0.01423773, 0.00714807, 0.00159793, ..., 0.00479249, 0.00591378,
       0.00115425])

In [79]:
xgb_pred

array([0.00653323, 0.0025562 , 0.00165315, ..., 0.03253714, 0.00376029,
       0.00088756], dtype=float32)

In [80]:
final_pred = (xgb_pred + lgbm_pred) / 2

In [81]:
final_pred = [1 if p >= 0.5 else 0 for p in final_pred]

In [82]:
submission = pd.DataFrame({'index' : np.arange(2500), 'Exited' : final_pred})

In [83]:
submission.Exited.value_counts()

0    2500
Name: Exited, dtype: int64

In [84]:
from sklearn.metrics import accuracy_score

In [85]:
accuracy_score(test['Exited'], final_pred)

0.8044

In [86]:
train.Exited.value_counts()

0    5952
1    1548
Name: Exited, dtype: int64

***

# 2번 문제

In [113]:
train = pd.read_csv('nyc_train.csv')
test = pd.read_csv('nyc_test.csv')

In [114]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65000 entries, 0 to 64999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   BLOCK                      65000 non-null  int64 
 1   LOT                        65000 non-null  int64 
 2   RESIDENTIAL UNITS          65000 non-null  int64 
 3   COMMERCIAL UNITS           65000 non-null  int64 
 4   TOTAL UNITS                65000 non-null  int64 
 5   LAND SQUARE FEET           65000 non-null  object
 6   GROSS SQUARE FEET          65000 non-null  object
 7   TAX CLASS AT TIME OF SALE  65000 non-null  int64 
 8   SALE PRICE                 65000 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 4.5+ MB


## 1. RESIDENTIAL UNITS을 정규화 한 후 0.7 보다 큰 데이터의 SALE PRICE 평균을 구하시오.

In [115]:
from sklearn.preprocessing import StandardScaler

In [116]:
scaler = StandardScaler()

In [117]:
scaled = scaler.fit_transform(train[['RESIDENTIAL UNITS']])

In [118]:
train['scaled_ru'] = scaled

In [119]:
train[train.scaled_ru > 0.7]['SALE PRICE'].mean()

10162272.97368421

## 2. MAE 기준으로 모델링 진행 후 예측값 도출 후 submission을 제출하시오.

In [120]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [121]:
train.drop('scaled_ru', axis = 1, inplace = True)

In [122]:
train.dtypes

BLOCK                         int64
LOT                           int64
RESIDENTIAL UNITS             int64
COMMERCIAL UNITS              int64
TOTAL UNITS                   int64
LAND SQUARE FEET             object
GROSS SQUARE FEET            object
TAX CLASS AT TIME OF SALE     int64
SALE PRICE                    int64
dtype: object

In [123]:
test.dtypes

BLOCK                         int64
LOT                           int64
RESIDENTIAL UNITS             int64
COMMERCIAL UNITS              int64
TOTAL UNITS                   int64
LAND SQUARE FEET             object
GROSS SQUARE FEET            object
TAX CLASS AT TIME OF SALE     int64
SALE PRICE                    int64
dtype: object

In [124]:
train['LAND SQUARE FEET'] = train['LAND SQUARE FEET'].str.replace(' -  ', '0').astype(float)
test['LAND SQUARE FEET'] = test['LAND SQUARE FEET'].str.replace(' -  ', '0').astype(float)

In [125]:
train['GROSS SQUARE FEET'] = train['GROSS SQUARE FEET'].str.replace(' -  ', '0').astype(float)
test['GROSS SQUARE FEET'] = test['GROSS SQUARE FEET'].str.replace(' -  ', '0').astype(float)

In [128]:
X = train.drop('SALE PRICE', axis = 1)
y = train['SALE PRICE']

target = test.drop('SALE PRICE', axis = 1)

In [129]:
rf_pred = np.zeros((target.shape[0]))
lgbm_pred = np.zeros((target.shape[0]))

In [130]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

#### RandomForest

In [134]:
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = RandomForestRegressor(random_state = 42)
    
    rf.fit(tr_x, tr_y)
    
    val_pred = rf.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    print(val_mae)
    
    fold_pred = rf.predict(target) / 5
    rf_pred += fold_pred

872529.2015056036
1039543.3330668787
898467.1590540019
953099.3934292006
862178.2394709871


#### LGBM

In [135]:
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMRegressor(random_state = 42)
    
    lgbm.fit(tr_x, tr_y)
    
    val_pred = lgbm.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    print(val_mae)
    
    fold_pred = lgbm.predict(target) / 5
    lgbm_pred += fold_pred

1106175.3153369897
1085249.5412559914
1021898.7398743805
1079403.538635477
972251.5650724671


In [136]:
final_pred = (rf_pred + lgbm_pred) / 2

In [137]:
test['prediction'] = final_pred

In [139]:
test[['SALE PRICE', 'prediction']]

,SALE PRICE,prediction
0,896500,6.413707e+05
1,780000,5.665862e+05
2,150000,3.241794e+05
3,3125000,3.369441e+06
4,564000,1.172668e+06
...,...,...
4982,1080000,6.381971e+05
4983,560000,4.649940e+05
4984,580000,4.647568e+05
4985,2337164,2.819118e+06


In [140]:
mean_absolute_error(test['SALE PRICE'], test['prediction'])

850514.7483358108